### Twitter Scaper and Sentiment Analysis in Spanish

1. Authentication with Twitter Developer account

In [4]:
import tweepy
import configparser
config = configparser.ConfigParser()
config.read("../config/keys_config.cfg")
api_key = config.get("twitter","api_key")
api_key_secret = config.get("twitter","api_key_secret")
access_token = config.get("twitter","access_token")
access_token_secret = config.get("twitter","access_token_secret")
bearer_token = config.get("twitter","bearer_token")

2.Function to receive tweets and save them to a pandas DataFrame

In [5]:
import requests
import pandas as pd
import numpy as np
import json

def get_tweets(query:str, n_results=1000):
    """
    Receives a string of twitter API rule as query for search_recent_tweets endpoint. Example: "carrefour argentina -is:retweet lang:es"
    Returns a pandas dataframe with text, created_ar, author_id and tweet_id columns.
    Requires imported tweepy, requests, pandas as pd, numpy as np, json
    """
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
    tweets = tweepy.Paginator(client.search_recent_tweets, query=query, tweet_fields=["created_at", "author_id"], max_results=100)
    df = pd.DataFrame(data=[[tweet.created_at, tweet.author_id, tweet.text] for tweet in tweets.flatten(limit=n_results)], columns=["Created_at","Author_id","Text"])
    return df

In [5]:
def clean_tweet(tweet: str):
    """
    Takes a text in string format.
    Returns this string without mentions, links, new line characters etc.
    """
    import re
    clean_tweet = tweet.replace("\n", " ")
    clean_tweet = re.sub("(@[a-zA-Z0-9][a-zA-Z0-9]+)(_[a-zA-Z0-9]+)*", " ", clean_tweet)
    clean_tweet = re.sub("(https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}([-a-zA-Z0-9()@:%_\+.~#?&//=]*))", " ", clean_tweet)
    clean_tweet = re.sub(" +", " ", clean_tweet)
    clean_tweet = clean_tweet.strip()
    return clean_tweet

def clean_df(df):
    df["Text"] = df["Text"].apply(clean_tweet)
    return df 
    
tweet_1 = clean_tweet("@jo_araos @LATAM_CHI Qué lata Josefina,pero veo que Latam te contestó.\nAprovecho de comentarles \nmi pésima experiencia con Assist Card ojo con esta empresa.\n 15 meses reclamado reembolso por gastos de Covid en https://t.co/lq7YjM3qmO oficina central la tienen en Argentina, de ahí te tramitan, pufff")
tweet_2 = clean_tweet("@opsoms @DrTedros  @opsomslas primeras inoculaciones")
tweet_3 = clean_tweet("Mas información en https://t.co/nTstpZsyrx o h...")
print(tweet_1)
print(tweet_2)
print(tweet_3)

Qué lata Josefina,pero veo que Latam te contestó. Aprovecho de comentarles mi pésima experiencia con Assist Card ojo con esta empresa. 15 meses reclamado reembolso por gastos de Covid en oficina central la tienen en Argentina, de ahí te tramitan, pufff
primeras inoculaciones
Mas información en o h...


In [6]:
from sentiment_analysis_spanish import sentiment_analysis
import matplotlib.pyplot as plt
def add_sentiment(df):
    sentiment = sentiment_analysis.SentimentAnalysisSpanish()
    df["sentiment"] = df["text"].apply(sentiment.sentiment)
    df["sent_label"] = df["sentiment"].apply(lambda x:"positivo" if x>=0.7 else ("neutro" if x>=0.3 and x<0.7 else "negativo"))
    sent_counts = df["sent_label"].value_counts()
    return df

def sentiment_count(df):
    print(df["sent_label"].value_counts())
    return df["sent_label"].plot.bar()


In [18]:
pd. set_option('display.max_colwidth', 1)
cristina = get_tweets("(Cristina Kirchner OR CFK) -is:retweet lang:es ", n_results=5000)
cristina = clean_df(cristina)
cristina.head()

,Created_at,Author_id,Text
0,2022-06-28 19:51:04+00:00,841026448513196032,"Como diría el 🦛: Te preparaste toda tu vida para ésto!!! Hint: Si CFK apoya a Massita ahora, es porque le hace ""la gran pichichi"". Lo manda al frente para que se incinere."
1,2022-06-28 19:50:47+00:00,1194807433,"Si quieren q los escuchen q les bloqueen las salidas de los domicilios de los q los llevaron a esto. MK el primero q hizo aprobar (2021) en el congreso la rebaja de 10a5% de biodiesel (soja)en la mezcla del gasoil para beneficiar a con la cámpora con YPF. Dps SM, CFK, AF. y de+"
2,2022-06-28 19:50:28+00:00,252245585,"De La A a la Z Reconocemos cada momento #Fan de #CFK? #Cristina ¡Aguante Cristina, manga de caretas¡"
3,2022-06-28 19:50:15+00:00,1378568534490505216,Como lo sabe hacer la falsa abogada de cfk.
4,2022-06-28 19:50:07+00:00,3170981873,"Así se expresó Juan Francisco Navarro, hijo del titular de la organización -""Chino"" Navarro-, en una fuerte carta contra la vicepresidenta. ""Nuestra responsabilidad es no abandonar a nuestro pueblo. No gritarles o seguir estigmatizándolos"", disparó."


In [22]:
from pysentimiento.preprocessing import preprocess_tweet
cristina["Text_prep"] = cristina["Text"].apply(preprocess_tweet)
cristina.to_csv("cristina_5000.csv")

In [23]:
planes = get_tweets("planes sociales lang:es -is:retweet", n_results = 5000)
planes["Text_prep"] = planes["Text"].apply(preprocess_tweet)
planes.to_csv("planes_5000.csv")

In [7]:
from pysentimiento.preprocessing import preprocess_tweet
planes_20mil = get_tweets("planes sociales lang:es -is:retweet", n_results=20000)
planes_20mil["Text_prep"] = planes_20mil["Text"].apply(preprocess_tweet)
planes_20mil.to_csv("planes_20.csv")